In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch import linalg as LA
import torch.optim as optim
from torchvision import datasets, transforms
from types import SimpleNamespace
import matplotlib.pyplot as plt
import numpy as np
from src import NeuralNet, SmoothSailing, kappa

In [2]:
args = SimpleNamespace(batch_size=32, test_batch_size=1000, epochs=10,
                       lr=0.0001, momentum=0.5, seed=1, log_interval=100, noise_level=0.1, beta=0.01)
torch.manual_seed(args.seed)
use_cuda = torch.cuda.is_available()
device = torch.device('cuda' if use_cuda else 'cpu')

In [3]:
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                     transform=transforms.Compose([
                            transforms.ToTensor(),
                            transforms.Normalize((0.1307,), (0.3081,))
                        ])),
    batch_size=args.batch_size, shuffle=True, **kwargs)
    
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=args.test_batch_size, shuffle=True, **kwargs)

In [4]:
model0 = NeuralNet(size=2048).to(device)
model0001 = NeuralNet(size=2048).to(device)
model001 = NeuralNet(size=2048).to(device)
model01 = NeuralNet(size=2048).to(device)
model1 = NeuralNet(size=2048).to(device)

In [5]:
# load models
m0 = torch.load('model_0_new.pt', map_location=torch.device('cpu'))
m0001 = torch.load('model_0001_new.pt', map_location=torch.device('cpu'))
m001 = torch.load('model_001_new.pt', map_location=torch.device('cpu'))
m01 = torch.load('model_01_new.pt', map_location=torch.device('cpu'))
m1 = torch.load('model_1_new.pt', map_location=torch.device('cpu'))

In [6]:
# load state_dicts into models
model0.load_state_dict(m0)
model0001.load_state_dict(m0001)
model001.load_state_dict(m001)
model01.load_state_dict(m01)
model1.load_state_dict(m1)

<All keys matched successfully>

In [10]:
model0.eval()
model0001.eval()
model001.eval()
model01.eval()
model1.eval()

acc0 = 0
acc0001 = 0
acc001 = 0
acc01 = 0
acc1 = 0

num_repeats = 100

for i in range(num_repeats):
    with torch.no_grad():
        for data, target in test_loader:
            # add noise
            #data += 1.5 * torch.randn_like(data)
            data = Variable(data.view(-1, 28*28))
            data, target = data.to(device), target.to(device)

            output0 = model0(data)
            output0001 = model0001(data)
            output001 = model001(data)
            output01 = model01(data)
            output1 = model1(data)

            pred0 = output0.max(1, keepdim=True)[1]
            accur0 = pred0.eq(target.view_as(pred0)).sum().item()
            pred0001 = output0001.max(1, keepdim=True)[1]
            accur0001 = pred0001.eq(target.view_as(pred0001)).sum().item()
            pred001 = output001.max(1, keepdim=True)[1]
            accur001 = pred001.eq(target.view_as(pred001)).sum().item()
            pred01 = output01.max(1, keepdim=True)[1]
            accur01 = pred01.eq(target.view_as(pred01)).sum().item()
            pred1 = output1.max(1, keepdim=True)[1]
            accur1 = pred1.eq(target.view_as(pred1)).sum().item()

        acc0 += accur0/args.test_batch_size*100
        acc0001 += accur0001/args.test_batch_size*100
        acc001 += accur001/args.test_batch_size*100
        acc01 += accur01/args.test_batch_size*100
        acc1 += accur1/args.test_batch_size*100

print(f"\tbeta 0: {acc0/num_repeats:.2f}%")
print(f"\tbeta 0.001: {acc0001/num_repeats:.2f}%")
print(f"\tbeta 0.01: {acc001/num_repeats:.2f}%")
print(f"\tbeta 0.1: {acc01/num_repeats:.2f}%")
print(f"\tbeta 1: {acc1/num_repeats:.2f}%")

# save the results
with open('cond_class_clean_2048.txt', 'w') as f:
    f.write(f"\tbeta 0: {acc0/num_repeats:.2f}%\n")
    f.write(f"\tbeta 0.001: {acc0001/num_repeats:.2f}%\n")
    f.write(f"\tbeta 0.01: {acc001/num_repeats:.2f}%\n")
    f.write(f"\tbeta 0.1: {acc01/num_repeats:.2f}%\n")
    f.write(f"\tbeta 1: {acc1/num_repeats:.2f}%\n")


	beta 0: 98.37%
	beta 0.001: 98.25%
	beta 0.01: 97.97%
	beta 0.1: 97.79%
	beta 1: 95.53%


In [ ]:
# load the pkl files
import pickle

with open('results_new.pkl', 'rb') as f:
    results = pickle.load(f)